<a href="https://colab.research.google.com/github/CookieNtoo/TestCode99/blob/main/IPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, LSTM, Flatten, Dropout
from keras.utils import to_categorical

# Φόρτωση δεδομένων
data = pd.read_csv('https://raw.githubusercontent.com/kdemertzis/EKPA/refs/heads/main/Data/pcap_data.csv')  # Αντικαταστήστε με το κατάλληλο αρχείο

# Προετοιμασία δεδομένων
X = data.drop('label', axis=1).values  # Χαρακτηριστικά
y = data['label'].values  # Ετικέτες

# Κωδικοποίηση ετικετών
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Διαχωρισμός δεδομένων σε εκπαιδευτικό και δοκιμαστικό σύνολο
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Αναδιάταξη των δεδομένων για Conv1D
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Δημιουργία του μοντέλου
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

# Συγκέντρωση του μοντέλου
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Εκπαίδευση του μοντέλου
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Αξιολόγηση του μοντέλου
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Λειτουργία ανίχνευσης και πρόληψης εισβολών
def detect_and_prevent_intrusion(new_data):
    new_data = new_data.reshape((new_data.shape[0], new_data.shape[1], 1))  # Αναδιάταξη
    prediction = model.predict(new_data)
    predicted_classes = np.argmax(prediction, axis=1)

    # Λογική απόφασης για την πρόληψη εισβολών
    actions = []
    for pred in predicted_classes:
        if pred == 1:  # Υποθέτουμε ότι η κατηγορία 1 είναι η κατηγορία εισβολής
            actions.append("Block traffic")  # Αποκλεισμός κυκλοφορίας
        elif pred == 2:  # Υποθέτουμε ότι η κατηγορία 2 είναι η κατηγορία κακόβουλης δραστηριότητας
            actions.append("Isolate device")  # Απομόνωση συσκευής
        else:
            actions.append("No action")  # Καμία ενέργεια

    return label_encoder.inverse_transform(predicted_classes), actions

# Παράδειγμα χρήσης
new_data = np.array([[...]])  # Αντικαταστήστε με νέα δεδομένα
results, actions = detect_and_prevent_intrusion(new_data)
print("Αποτελέσματα ανίχνευσης:", results)
print("Ενέργειες:", actions)

KeyError: "['label'] not found in axis"